In [1]:
import hddm
import pandas as pd
import numpy as np
import pickle

In [2]:
import os

In [65]:
modelName = 'ep5'
m = hddm.load('savedModels/{}/{}'.format(modelName, modelName))

In [66]:
m.print_stats()

                         mean       std      2.5q       25q       50q       75q     97.5q    mc err
a                    2.296469  0.073716  2.159938  2.247548  2.294192  2.343537  2.445431  0.002074
a_std                0.463373  0.060892   0.35954  0.420963    0.4572  0.500959   0.60326  0.001779
a_subj.1             2.338602  0.130626  2.098486  2.249451  2.335693  2.424153  2.603995  0.003565
a_subj.2             1.982984  0.129587   1.74486  1.893346  1.980423  2.069959  2.245924  0.004506
a_subj.3             3.033928   0.19837  2.668435  2.898547  3.026693   3.15712  3.450593   0.00586
a_subj.4             2.125917  0.115856  1.918885  2.046307  2.120514  2.198391  2.377818  0.003379
a_subj.5             1.900796  0.296195  1.399464  1.688946  1.873166  2.076409  2.543422  0.010807
a_subj.6             2.309437  0.276743  1.891292  2.118151  2.270845  2.448839  2.986168  0.014929
a_subj.7             2.229103   0.27604  1.756688   2.04311  2.207613  2.386649  2.825192  0.014016


In [67]:
import numpy as np
modelParamsDF = m.nodes_db
modelParamsDF.to_csv('savedModels/{}/{}_nodes.csv'.format(modelName, modelName))


In [68]:
# group level mean of lambda
groupLevelMeanOfLambda = modelParamsDF.loc['v_loss', 'mean']/modelParamsDF.loc['v_gain', 'mean']
print("group level mean of lamdba is: ", round(groupLevelMeanOfLambda, 3))
# 95% CI for beta_G, beta_L
print("2.5% percentile for beta_L: ", round(modelParamsDF.loc['v_loss', '2.5q'], 3))
print("97.5% percentile for beta_G: ", round(modelParamsDF.loc['v_gain', '97.5q'], 3))

group level mean of lamdba is:  1.784
2.5% percentile for beta_L:  0.016
97.5% percentile for beta_G:  0.016


In [69]:
# how many participants have beta_L > beta_G, how many have the 95% CI greater?
# participant level mean (SD) of lambda
numSubjects = 49
numLambdaMean = 0
numLambdaCI = 0
allParticipantLambdas = []

for subjectIndex in range(1, numSubjects+1):
    try:
        beta_gain = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), 'mean']
        beta_loss = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), 'mean']
        beta_L_lowerBound = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), '2.5q']
        beta_G_upperBound = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), '97.5q']

        if beta_loss > beta_gain:
            numLambdaMean += 1
        if beta_L_lowerBound > beta_G_upperBound:
            numLambdaCI += 1

        lambdaForThisParticipant = beta_loss / beta_gain
        allParticipantLambdas.append(lambdaForThisParticipant)

    except KeyError:
        print(f"Data for participant {subjectIndex} is missing.")
        continue  # Skip the rest of the loop and move to the next iteration

print("Number of people with beta_L > beta_G = ", round(numLambdaMean, 3))
print("Number of people with 95% CI of beta_L > beta_G = ", numLambdaCI)
print("Average lambda over all individuals = {} (SD = {})".format(round(np.mean(allParticipantLambdas), 3), round(np.std(allParticipantLambdas), 3)))


Number of people with beta_L > beta_G =  37
Number of people with 95% CI of beta_L > beta_G =  11
Average lambda over all individuals = 5.552 (SD = 41.03)


In [70]:
import numpy as np

numSubjects = 49
numLambdaMean = 0
numLambdaCI = 0
allParticipantLambdas = []

# Lists to store the indices of the participants meeting the conditions
participantsWithBetaLGreater = []
participantsWith95CIGreater = []

for subjectIndex in range(1, numSubjects+1):
    try:
        beta_gain = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), 'mean']
        beta_loss = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), 'mean']
        beta_L_lowerBound = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), '2.5q']
        beta_G_upperBound = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), '97.5q']
    
        if beta_loss > beta_gain:
            numLambdaMean += 1
            participantsWithBetaLGreater.append(subjectIndex)
        if beta_L_lowerBound > beta_G_upperBound:
            numLambdaCI += 1
            participantsWith95CIGreater.append(subjectIndex)
        
        lambdaForThisParticipant = beta_loss/beta_gain
        allParticipantLambdas.append(lambdaForThisParticipant)
        
    except KeyError:
        print(f"Data for participant {subjectIndex} is missing.")
        continue  # Skip the rest of the loop and move to the next iteration
print("Number of people with beta_L > beta_G = ", round(numLambdaMean, 3))
print("Participants with beta_L > beta_G: ", participantsWithBetaLGreater)
print("Number of people with 95% CI of beta_L > beta_G = ", numLambdaCI)
print("Participants with 95% CI of beta_L > beta_G: ", participantsWith95CIGreater)
print("Average lambda over all individuals = {} (SD = {})".format(round(np.mean(allParticipantLambdas), 3), round(np.std(allParticipantLambdas), 3)))


Number of people with beta_L > beta_G =  37
Participants with beta_L > beta_G:  [1, 2, 3, 4, 5, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 30, 32, 34, 35, 36, 38, 39, 41, 43, 44, 45, 46, 47, 48, 49]
Number of people with 95% CI of beta_L > beta_G =  11
Participants with 95% CI of beta_L > beta_G:  [3, 10, 18, 21, 22, 23, 25, 34, 41, 43, 47]
Average lambda over all individuals = 5.552 (SD = 41.03)


In [71]:
# how many participants have beta_L > beta_G, how many have the 95% CI greater?
# participant level mean (SD) of lambda
numSubjects = 49
numLambdaMean = 0
numLambdaCI = 0
allParticipantLambdas = []

for subjectIndex in range(1, numSubjects+1):
    try:
        beta_gain = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), 'mean']
        beta_loss = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), 'mean']
        
        # Skip if either beta_gain or beta_loss is negative
        if beta_gain <= 0 or beta_loss <= 0:
            continue

        beta_L_lowerBound = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), '2.5q']
        beta_G_upperBound = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), '97.5q']

        if beta_loss > beta_gain:
            numLambdaMean += 1
        if beta_L_lowerBound > beta_G_upperBound:
            numLambdaCI += 1

        lambdaForThisParticipant = beta_loss / beta_gain
        allParticipantLambdas.append(lambdaForThisParticipant)

    except KeyError:
        print(f"Data for participant {subjectIndex} is missing.")
        continue  # Skip the rest of the loop and move to the next iteration

print("Number of people with beta_L > beta_G = ", round(numLambdaMean, 3))
print("Number of people with 95% CI of beta_L > beta_G = ", numLambdaCI)
print("Average lambda over all individuals = {} (SD = {})".format(round(np.mean(allParticipantLambdas), 3), round(np.std(allParticipantLambdas), 3)))


Number of people with beta_L > beta_G =  25
Number of people with 95% CI of beta_L > beta_G =  5
Average lambda over all individuals = 11.04 (SD = 49.389)


In [72]:
print("beta_L mean:", round(modelParamsDF.loc['v_loss', 'mean'], 3))
print("beta_L 2.5q:", round(modelParamsDF.loc['v_loss', '2.5q'], 3))
print("beta_L 97.5q:", round(modelParamsDF.loc['v_loss', '97.5q'], 3))
print("beta_L median", round(modelParamsDF.loc['v_loss', '50q'], 3))
print("beta_L SD", round(modelParamsDF.loc['v_loss', 'std'], 3))

print("beta_G mean:", round(modelParamsDF.loc['v_gain', 'mean'], 3))
print("beta_G 2.5q:", round(modelParamsDF.loc['v_gain', '2.5q'], 3))
print("beta_G 97.5q:", round(modelParamsDF.loc['v_gain', '97.5q'], 3))
print("beta_G median", round(modelParamsDF.loc['v_gain', '50q'], 3))
print("beta_G SD", round(modelParamsDF.loc['v_gain', 'std'], 3))

beta_L mean: 0.02
beta_L 2.5q: 0.016
beta_L 97.5q: 0.025
beta_L median 0.02
beta_L SD 0.002
beta_G mean: 0.011
beta_G 2.5q: 0.006
beta_G 97.5q: 0.016
beta_G median 0.011
beta_G SD 0.003


In [73]:
meanA = modelParamsDF.loc['a', 'mean']
print("mean value of threshold: ", meanA)

# 95 CI interval
Z_CI_lowerBound = (modelParamsDF.loc['z'.format(subjectIndex), '2.5q'] - 0.5)*2
Z_CI_upperBound = (modelParamsDF.loc['z'.format(subjectIndex), '97.5q'] - 0.5)*2
print("confidence interval of gamma: ", round(Z_CI_lowerBound, 3), round(Z_CI_upperBound, 3))

mean value of threshold:  2.2964690439465207
confidence interval of gamma:  -0.184 -0.079


In [74]:
numSubjects = 49
allParticipantsZ = []
countGammaMean = 0
countGammaCI = 0

for participantIndex in range(1, numSubjects+1):
    try:
        a_participant = modelParamsDF.loc['a_subj.{}'.format(participantIndex), 'mean']
        z_participant = (modelParamsDF.loc['z_subj.{}'.format(participantIndex), 'mean'] - 0.5)*2
        allParticipantsZ.append(z_participant)
        z_upperBound_participant = (modelParamsDF.loc['z_subj.{}'.format(participantIndex), '97.5q'] - 0.5)*2
    
        if z_upperBound_participant < 0:
            countGammaCI += 1
        
        if z_participant < 0:
            countGammaMean += 1
    
    except KeyError:
        print(f"Data for participant {participantIndex} is missing.")
        continue  # Skip the rest of the loop and move to the next iteration

# Calculate the mean and standard deviation after collecting all participant data
mean_Z = np.mean(allParticipantsZ)
SD_Z = np.std(allParticipantsZ)

print("Mean value of pre-valuation bias (gamma) across all participants: {} (SD = {})".format(round(mean_Z, 3), round(SD_Z, 3)))
print("Number of participants with negative gamma:", countGammaMean)
print("Number of participants with 95% CI of gamma negative: ", round(countGammaCI, 3))


Mean value of pre-valuation bias (gamma) across all participants: -0.129 (SD = 0.174)
Number of participants with negative gamma: 38
Number of participants with 95% CI of gamma negative:  23


In [75]:
print("gamma mean:", round((modelParamsDF.loc['z', 'mean'] - 0.5)*2, 3))
print("gamma 2.5q:", round((modelParamsDF.loc['z', '2.5q'] - 0.5)*2, 3))
print("gamma 97.5q:", round((modelParamsDF.loc['z', '97.5q'] - 0.5)*2, 3))
print("gamma median", round((modelParamsDF.loc['z', '50q'] - 0.5)*2, 3))
print("gamma SD", round((modelParamsDF.loc['z', 'std'])*2, 3))

gamma mean: -0.132
gamma 2.5q: -0.184
gamma 97.5q: -0.079
gamma median -0.131
gamma SD 0.027


In [76]:
numSubjects = 49
allParticipantsAlphas = []
countAlphaMean = 0
countAlphaCI = 0

for participantIndex in range(1, numSubjects+1):
    try:
        alpha_participant = modelParamsDF.loc['v_Intercept_subj.{}'.format(participantIndex), 'mean']
        allParticipantsAlphas.append(alpha_participant)
        alpha_upperBound_participant = modelParamsDF.loc['v_Intercept_subj.{}'.format(participantIndex), '97.5q']
    
        if alpha_upperBound_participant < 0:
            countAlphaCI += 1
        
        if alpha_participant < 0:
            countAlphaMean += 1
    
    except KeyError:
        print(f"Data for participant {participantIndex} is missing.")
        continue  # Skip the rest of the loop and move to the next iteration

# Calculate the mean and standard deviation after collecting all participant data
mean_alpha = np.mean(allParticipantsAlphas)
SD_alpha = np.std(allParticipantsAlphas)

print("Mean value of fixed utility bias across all participants: {} (SD = {})".format(round(mean_alpha, 3), round(SD_alpha, 3)))
print("Number of participants with negative alpha: ", round(countAlphaMean, 3))
print("Number of participants with 95% CI of alpha negative: ", round(countAlphaCI, 3))

# Print group-level mean of alpha
group_level_mean_alpha = modelParamsDF.loc['v_Intercept', 'mean']
print("Group-level mean of alpha is: ", round(group_level_mean_alpha, 3))


Mean value of fixed utility bias across all participants: -0.416 (SD = 0.962)
Number of participants with negative alpha:  30
Number of participants with 95% CI of alpha negative:  14
Group-level mean of alpha is:  -0.412


In [77]:
print("alpha mean:", round(modelParamsDF.loc['v_Intercept', 'mean'], 3))
print("alpha 2.5q:", round(modelParamsDF.loc['v_Intercept', '2.5q'], 3))
print("alpha 97.5q:", round(modelParamsDF.loc['v_Intercept', '97.5q'], 3))
print("alpha median", round(modelParamsDF.loc['v_Intercept', '50q'], 3))
print("alpha SD", round(modelParamsDF.loc['v_Intercept', 'std'], 3))

alpha mean: -0.412
alpha 2.5q: -0.729
alpha 97.5q: -0.089
alpha median -0.409
alpha SD 0.163


In [78]:
print("theta mean:", round(modelParamsDF.loc['a', 'mean']/2, 3))
print("theta 2.5q:", round(modelParamsDF.loc['a', '2.5q']/2, 3))
print("theta 97.5q:", round(modelParamsDF.loc['a', '97.5q']/2, 3))
print("theta median", round(modelParamsDF.loc['a', '50q']/2, 3))
print("theta SD", round(modelParamsDF.loc['a', 'std']/2, 3))

theta mean: 1.148
theta 2.5q: 1.08
theta 97.5q: 1.223
theta median 1.147
theta SD 0.037


In [79]:
print("tau mean:", round(modelParamsDF.loc['t', 'mean'], 3))
print("tau 2.5q:", round(modelParamsDF.loc['t', '2.5q'], 3))
print("tau 97.5q:", round(modelParamsDF.loc['t', '97.5q'], 3))
print("tau median", round(modelParamsDF.loc['t', '50q'], 3))
print("tau SD", round(modelParamsDF.loc['t', 'std'], 3))

tau mean: 0.434
tau 2.5q: 0.397
tau 97.5q: 0.473
tau median 0.434
tau SD 0.019
